# Key Points:

1. In this notebook I will create a RAG system
2. We'll be using Chroma DB
3. We'll store db on local and reload without re-processing the documents

In [27]:
!pip -q install langchain openai tiktoken chromadb

In [28]:
!pip show langchain

Name: langchain
Version: 0.0.340
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [29]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
!unzip -q new_articles.zip -d new_articles

replace new_articles/05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass()   # OPENAI api key

··········


In [2]:
from langchain import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, DirectoryLoader


## Load and process multiple documents

In [3]:
# loader = TextLoader("single_text_file.txt")
loader = DirectoryLoader(path="/content/new_articles", glob = "./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [4]:
# Splitting the text into text chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

In [5]:
texts = text_splitter.split_documents(documents)

In [6]:
len(texts)

233

In [7]:
texts[:2]

[Document(page_content='SpaceX’s super-heavy launch system Starship is poised to fundamentally reshape the space economy. The 394-foot-tall vehicle, which took to the skies for the first time last month, is designed to carry a staggering amount of mass to low Earth orbit and into deep space.\n\nTechCrunch+ spoke with three pure-play space VCs — Space Capital founder and managing partner Chad Anderson, Space.VC founder and general partner Jonathan Lacoste and E2MC Ventures founder Raphael Roettgen — to learn more about how they advise founders to think through Starship’s super-heavy implications.\n\nWhile the trio diverges on many fine points, they all agreed that founders should be thinking now about how Starship could affect their operations, for better or worse.\n\n“Starship has such high importance to the space sector that probably almost everyone who has a space company has to war game what that means for their business,” Roettgen said.\n\nChanging the face of launch …', metadata={

## Create the DB

In [8]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk

persist_directory = "db"

# here we are using OpenAI embeddings
embedding = OpenAIEmbeddings()

vector_db = Chroma.from_documents(documents = texts,
                                  embedding = embedding,
                                  persist_directory = persist_directory)

In [9]:
# persist db to the disk
vector_db.persist()
vector_db = None

In [10]:
# Now we can load the persisted database from disk, and use it as normal
vector_db = Chroma(persist_directory = persist_directory,
                   embedding_function = embedding)

## Make a Retriever

In [11]:
retriever = vector_db.as_retriever()
docs = retriever.get_relevant_documents("How much money Pando made?")

In [14]:
docs

[Document(page_content='Signaling that investments in the supply chain sector remain robust, Pando, a startup developing fulfillment management technologies, today announced that it raised $30 million in a Series B round, bringing its total raised to $45 million.\n\nIron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.\n\n“We will not expand into new industries or adjacent product areas,” he told TechCrunch in an email interview. “Great talent is the foundation of the business — we will continue to augment our teams at all levels of the organization. Pando is also open to exploring strategic partnerships and acquisitions with this round of funding.”', metadata={'source': '/content/new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30

In [15]:
retriever = vector_db.as_retriever(search_kwargs = {"k":2})
docs = retriever.get_relevant_documents("How much money Pando made?")
docs

[Document(page_content='Signaling that investments in the supply chain sector remain robust, Pando, a startup developing fulfillment management technologies, today announced that it raised $30 million in a Series B round, bringing its total raised to $45 million.\n\nIron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.\n\n“We will not expand into new industries or adjacent product areas,” he told TechCrunch in an email interview. “Great talent is the foundation of the business — we will continue to augment our teams at all levels of the organization. Pando is also open to exploring strategic partnerships and acquisitions with this round of funding.”', metadata={'source': '/content/new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30

In [16]:
retriever.search_type

'similarity'

In [17]:
retriever.search_kwargs

{'k': 2}

## Make a Chain

In [18]:
# Create a chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    chain_type= "stuff",
    llm = OpenAI(),
    retriever = retriever,
    return_source_documents = True
)


In [30]:
# Cite sources
def process_llm_response(llm_response):
  print(llm_response['result'])
  print("\n\nSources: ")
  for src_documents in llm_response['source_documents']:
    print(src_documents.metadata['source'])

In [20]:
query = "How much money Pando made?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Pando made $45 million.


Sources: 
/content/new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
/content/new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [31]:
# break it down
query = "What is the news about Pando?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'What is the news about Pando?',
 'result': ' Pando is a software-as-a-service platform offering created in 2018 by Jayakrishnan and Abhijeet Manohar. It was designed to help manufacturers, distributors, and retailers understand, optimize, and manage their global logistics operations.',
 'source_documents': [Document(page_content='Pando was co-launched by Jayakrishnan and Abhijeet Manohar, who previously worked together at iDelivery, an India-based freight tech marketplace — and their first startup. The two saw firsthand manufacturers, distributors and retailers were struggling with legacy tech and point solutions to understand, optimize and manage their global logistics operations — or at least, that’s the story Jayakrishnan tells.\n\n“Supply chain leaders were trying to build their own tech and throwing people at the problem,” he said. “This caught our attention — we spent months talking to and building for enterprise users at warehouses, factories, freight yards and ports 

In [32]:
query = "Who led the round in Pando?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Iron Pillar and Uncorrelated Ventures led the round.


Sources: 
/content/new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
/content/new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [33]:
query = "What did databricks acquire?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Databricks acquired Okera.


Sources: 
/content/new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt
/content/new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt


In [34]:
query = "What is generative ai?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Generative AI is a type of artificial intelligence that uses data to create new content, such as text, images, or music. It is used to create content that is similar to what a human would create.


Sources: 
/content/new_articles/05-04-slack-updates-aim-to-put-ai-at-the-center-of-the-user-experience.txt
/content/new_articles/05-04-slack-updates-aim-to-put-ai-at-the-center-of-the-user-experience.txt


In [35]:
query = "Who is CMA?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 CMA stands for the Competition and Markets Authority.


Sources: 
/content/new_articles/05-04-cma-generative-ai-review.txt
/content/new_articles/05-04-cma-generative-ai-review.txt


In [36]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7d0e109bae90>)

In [37]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Deleting the db

In [39]:
!zip -r db.zip ./db # Zipping everything present in db directory

updating: db/ (stored 0%)
updating: db/2b98eed9-7219-4a0d-b52e-be05e11b22eb/ (stored 0%)
updating: db/2b98eed9-7219-4a0d-b52e-be05e11b22eb/header.bin (deflated 61%)
updating: db/2b98eed9-7219-4a0d-b52e-be05e11b22eb/link_lists.bin (stored 0%)
updating: db/2b98eed9-7219-4a0d-b52e-be05e11b22eb/length.bin (deflated 99%)
updating: db/2b98eed9-7219-4a0d-b52e-be05e11b22eb/data_level0.bin (deflated 100%)
updating: db/chroma.sqlite3 (deflated 39%)


In [41]:

# To cleanup, you can delete the collection
vector_db.delete_collection()
vector_db.persist()



In [42]:
# delete the directory
!rm -rf db/

## Loading db again

restart the runtime so everything is cleaned from the memory

In [1]:
!unzip db.zip # unzipping the zipped db

Archive:  db.zip
   creating: db/
   creating: db/2b98eed9-7219-4a0d-b52e-be05e11b22eb/
  inflating: db/2b98eed9-7219-4a0d-b52e-be05e11b22eb/header.bin  
 extracting: db/2b98eed9-7219-4a0d-b52e-be05e11b22eb/link_lists.bin  
  inflating: db/2b98eed9-7219-4a0d-b52e-be05e11b22eb/length.bin  
  inflating: db/2b98eed9-7219-4a0d-b52e-be05e11b22eb/data_level0.bin  
  inflating: db/chroma.sqlite3       


In [2]:
  import os
  import getpass

  os.environ['OPENAI_API_KEY'] = getpass.getpass()

··········


In [3]:
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

In [4]:
persist_directory = "db"
embedding = OpenAIEmbeddings()
vector_db_2 = Chroma(
    persist_directory = persist_directory,
    embedding_function = embedding,
)

In [5]:
retriever = vector_db_2.as_retriever(search_kwargs = {'k':2})

In [6]:
# Setting up OpenAI chat llm
llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0,
)

In [7]:
# Create the chain to answer question

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True
)

In [10]:
def process_llm_response(llm_response):
  print(llm_response['result'])
  print("\nSource Documents: ")
  for src_docs in llm_response['source_documents']:
    print(src_docs.metadata['source'])

In [12]:
query = "How much money did Pando raise?"
llm_response = chain(query)
process_llm_response(llm_response)

Pando raised $30 million in its Series B round, bringing its total raised to $45 million.

Source Documents: 
/content/new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
/content/new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


## Chat Prompt

In [13]:
print(chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [14]:
print(chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}


In [19]:
print(chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
